In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support.ui import Select

from bs4 import BeautifulSoup
import re
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO

pd.set_option('display.expand_frame_repr', False)

In [ ]:
# Define a custom expected condition to handle StaleElementReferenceException
def custom_wait_func(refreshed_locator):
    def check_for_elem(driver):
        try:
            element = driver.find_element(*refreshed_locator)
            return element
        except:
            return False

    return check_for_elem


def find_els_wait_func(refreshed_locator):
    def check_for_elem(driver):
        try:
            element = driver.find_elements(*refreshed_locator)
            return element
        except:
            return False

    return check_for_elem

In [ ]:
#Initialize Chrome Webdriver
driver = webdriver.Chrome()

url = "https://efdsearch.senate.gov/search/"  
driver.get(url)

#Read the data as HTML
html_content = driver.page_source
agree_checkbox = driver.find_element(By.ID, "agree_statement")

# Check if the checkbox is not already selected
if not agree_checkbox.is_selected():
    # If the checkbox is not selected, click on it
    agree_checkbox.click()


#select current senators
senator_checkbox = driver.find_element(By.CLASS_NAME,"form-check-input")

if not senator_checkbox.is_selected():
    # If the checkbox is not selected, click on it
    senator_checkbox.click()
    
#select prev senators
prev_senators_checkbox = driver.find_element(By.XPATH, "//input[@value='5']")
if not prev_senators_checkbox.is_selected():
    # If the checkbox is not selected, click on it
    prev_senators_checkbox.click()

#Select annual and periodic transaction reports
annual_checkbox = driver.find_element(By.XPATH, "//input[@value='7']")
if not annual_checkbox.is_selected():
    # If the checkbox is not selected, click on it
    annual_checkbox.click()


#insert starting year of 2013 (most of 2012 reports are written reports and cant be scraped

start_date_box = driver.find_element(By.XPATH, "//input[@name='submitted_start_date']")
start_date_box.send_keys("01/01/2013")


#hit submit button
submit_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH,"//button[@class='btn btn-primary']"))
)
submit_button.click()


#at this point we are on a page that has all former and current senator annual/periodic filings
#dating back to 1/13/2013

#we know need to iterate through each of the 1,704 filings, check if it is an image, in which case we 
#skip because we cant read


#first sort by ascending order
sort_data_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH,"//th[@aria-label='Date Received/Filed: activate to sort column ascending']"))
)
sort_data_button.click()
#make it descending
sort_data_button = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.XPATH,"//th[@aria-label='Date Received/Filed: activate to sort column descending']"))
)
sort_data_button.click()

#make it 100 entries per page
reports_length = driver.find_element(By.XPATH,"//select[@name='filedReports_length']")
reports_length_select = Select(reports_length)
reports_length_select.select_by_value("100")



time.sleep(2)






#There are 17 pages of reports, with 100 reports per page
#This script will save a csv of each page.
#if your connection is interrupted
#insert the most recent saved page
#ie, if the connection is interrupted on page 4, put 3 as the checkpoint
checkpoint = 0

#skip pages up to checkpoint
for i in range(checkpoint):
    driver.execute_script("window.scrollTo(0, 0)")
    table = WebDriverWait(driver, 10).until(
        custom_wait_func((By.XPATH, "//table[@class='table table-striped dataTable no-footer']"))
    )
    next_button = WebDriverWait(driver, 10).until(
        custom_wait_func((By.XPATH, "//a[@class='paginate_button next']"))
    )
    next_button.click()


#Iterate over reports
for i in range(checkpoint, 17): 
    #filings
    senator_filings_list = []
    senator_name = []
    #scroll to top of page
    driver.execute_script("window.scrollTo(0, 0)")
    table = WebDriverWait(driver, 10).until(
        custom_wait_func((By.XPATH, "//table[@class='table table-striped dataTable no-footer']"))
    )

    rows = WebDriverWait(table, 10).until(
        find_els_wait_func((By.TAG_NAME, "tr"))
    )
    
    
    # table.find_elements(By.TAG_NAME, "tr")
    # #iterate over each link to each senator report
    for j,row in enumerate(rows):
        table = WebDriverWait(driver, 10).until(
            custom_wait_func((By.XPATH, "//table[@class='table table-striped dataTable no-footer']"))
        )
        row = table.find_elements(By.TAG_NAME, "tr")[j]
        cells = row.find_elements(By.TAG_NAME, "td")
        if (len(cells)>4):
            current_window_handle = driver.current_window_handle

            link = cells[3].find_element(By.TAG_NAME, "a")
            link.click()
            new_window_handle = [handle for handle in driver.window_handles if handle != current_window_handle][0]
            driver.switch_to.window(new_window_handle)
            
            #wait for page to load
            WebDriverWait(driver, 10).until(
                    custom_wait_func((By.XPATH, "//main[@class='container-fluid pgContent']"))
            )



            #if the page is not a handwritten for proceed
            if not driver.find_elements(By.XPATH, "//img[@class='filingImage']"):
                html_content = driver.page_source
                soup = BeautifulSoup(html_content, 'html.parser')

                #get Senator name
                name = driver.find_element(By.XPATH, "//h2[@class='filedReport']")
                pattern = r"\((.*?)\)"
                
                # Find all matches of the pattern in the text
                matches = re.findall(pattern, name.text)
                if (len(matches) > 0 and matches[0] != "Former Senator"):
                    name = matches[0]
                else:
                    name = name.text
                    
                print(name)
                
                #get all tables
                #only interested in part 4a and 4b
                sections = soup.find_all('section')

                if (len(sections) >= 5):
                    table_section_part4a = sections[3]
                    table_4a = table_section_part4a.find('table')
                        # If the table is found, read it into a DataFrame
                    if table_4a:
                        table_4a = str(table_4a)
                        table_4a = StringIO(table_4a)
                        senator_filings_list.append(table_4a)
                        senator_name.append(name)
    

                    table_section_part4b = sections[4]
                    table_4b = table_section_part4b.find('table')
                        # If the table is found, read it into a DataFrame
                    if table_4b:
                        table_4b = str(table_4b)
                        table_4b = StringIO(table_4b)
                        senator_filings_list.append(table_4b)
                        senator_name.append(name)

            
            driver.close()
            
            # Switch back to the original tab
            driver.switch_to.window(current_window_handle)

    #every page create df a save in case we are disconnected
    dataframes = []
    for name,table in zip(senator_name,senator_filings_list):
        df = pd.read_html(table)[0]  # Read the HTML table into a DataFrame
        df['Senator Name'] = name
        dataframes.append(df)
    
    # Concatenate all DataFrames into a single DataFrame
    result_df = pd.concat(dataframes, ignore_index=True)
    filename= f'senator_filings_page_{i}.csv'
    result_df.to_csv(filename)
    
    
    next_button = WebDriverWait(driver, 10).until(
        custom_wait_func((By.XPATH, "//a[@class='paginate_button next']"))
    )
    next_button.click()


# dataframes = []
# for name,table in zip(senator_name,senator_filings_list):
#     df = pd.read_html(table)[0]  # Read the HTML table into a DataFrame
#     df['Senator Name'] = name
#     dataframes.append(df)

# # Concatenate all DataFrames into a single DataFrame
# result_df = pd.concat(dataframes, ignore_index=True)

# Print the result DataFrame
 


# senator_filing_data_df = result_df.copy()
driver.quit()


#### Data Now Collected
Now that we've scraped each page, converted to csv, and saved, we can now read

In [289]:
import glob

# List all CSV files in the directory
csv_files = glob.glob('*.csv')

dfs = []

# Iterate over each CSV file
for csv_file in csv_files:
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    # Append the DataFrame to the list
    dfs.append(df)


senator_filing_data_df = pd.concat(dfs, ignore_index=True)


## Clean The Data

In [290]:
pd.set_option('display.max_rows', None)

#drop unneccessary columns
senator_filing_data_df.drop(['Unnamed: 0','Unnamed: 0.1', '#'],axis=1, inplace=True)
#merge Transaction Type and Type as they are the same
senator_filing_data_df['merged_type'] = senator_filing_data_df['Type'].fillna(senator_filing_data_df['Transaction Type'])

#drop unneccessary columns
senator_filing_data_df.drop(['Type', 'Transaction Type', 'Comment', 'Comments'], axis=1, inplace=True)
#convert Transaction Date to Datetime 

# Rename the merged column to 'type' if needed
senator_filing_data_df.rename(columns={'merged_type': 'Transaction Type'}, inplace=True)
senator_filing_data_df.head(5)


,Transaction Date,Owner,Ticker,Asset Name,Amount,Senator Name,Transaction Type
0,04/29/2014,Joint,LINE,"Linn Energy, LLC (NASDAQ)","$1,001 - $15,000",The Honorable Patrick J Toomey (Former Senator),Sale (Full)
1,06/05/2014,Joint,GLD,SPDR Gold Shares,"$1,001 - $15,000",The Honorable Patrick J Toomey (Former Senator),Sale (Partial)
2,06/05/2014,Joint,PPL,PPL Corporation (NYSE),"$15,001 - $50,000",The Honorable Patrick J Toomey (Former Senator),Sale (Partial)
3,06/10/2014,Self,--,Team Capitol Bank,"$500,001 - $1,000,000",The Honorable Patrick J Toomey (Former Senator),Exchange
4,06/11/2014,Joint,LGP,Lehigh Gas Partners LP (NYSE),"$15,001 - $50,000",The Honorable Patrick J Toomey (Former Senator),Purchase


#### Convert 'Transaction Date' To Datetime
Here we'll want to turn the 'Transaction Date' column to datetime to make it easier for future calculations
and sorting

In [291]:
senator_filing_data_df['Transaction Date'] = pd.to_datetime(senator_filing_data_df['Transaction Date'])



ValueError: time data "02/21/217" doesn't match format "%m/%d/%Y", at position 686. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

That didnt work, it looks like theres at typo "02/21/217", where it should be "02/21/2017." Lets see how many times this happens, and where they occur. 

In [292]:
error_date_col = senator_filing_data_df[senator_filing_data_df['Transaction Date'].str.len() != 10]
error_date_col.head(5)

,Transaction Date,Owner,Ticker,Asset Name,Amount,Senator Name,Transaction Type
4182,02/21/217,Self,JALBX,JHancock Multimanager Lifestyle Bal A,"$100,001 - $250,000",The Honorable Michael B Enzi (Former Senator),Sale (Full)
39984,02/28/218,Spouse,PRWCX,T. Rowe Price Capital Appreciation Fund,"$1,001 - $15,000","Cassidy, Bill",Sale (Partial)
40013,07/06/218,Spouse,JNBSX,JPMorgan Income Builder Fund Class I Shares,"$1,001 - $15,000","Cassidy, Bill",Sale (Partial)
40806,02/28/218,Spouse,PRWCX,T. Rowe Price Capital Appreciation Fund,"$1,001 - $15,000","Cassidy, Bill",Sale (Partial)
40835,07/06/218,Spouse,JNBSX,JPMorgan Income Builder Fund Class I Shares,"$1,001 - $15,000","Cassidy, Bill",Sale (Partial)


lets change them!

In [293]:
mask = senator_filing_data_df['Transaction Date'].str.len() != 10
# Change the entries in the original DataFrame based on the mask
senator_filing_data_df.loc[mask, 'Transaction Date'] = senator_filing_data_df.loc[mask, 'Transaction Date'].apply(lambda x: x[:7] + '0' + x[7:])

In [294]:
error_date_col = senator_filing_data_df[senator_filing_data_df['Transaction Date'].str.len() != 10]
error_date_col.shape

(0, 7)

In [295]:
#Double check 0 was added
senator_filing_data_df.loc[4182]

Transaction Date                                       02/21/2017
Owner                                                        Self
Ticker                                                      JALBX
Asset Name                  JHancock Multimanager Lifestyle Bal A
Amount                                        $100,001 - $250,000
Senator Name        The Honorable Michael B Enzi (Former Senator)
Transaction Type                                      Sale (Full)
Name: 4182, dtype: object

#### Looks good
Now we can convert

In [296]:
senator_filing_data_df['Transaction Date'] = pd.to_datetime(senator_filing_data_df['Transaction Date'])

Lets try sorting by date in ascending order

In [297]:
sorted_filings = senator_filing_data_df.sort_values(by='Transaction Date', ascending=True)
sorted_filings.head(6)

,Transaction Date,Owner,Ticker,Asset Name,Amount,Senator Name,Transaction Type
30104,2002-06-28,Self,JEPI,JPM Equity Premium Income ETF,"$1,001 - $15,000","Peters, Gary",Purchase
34628,2002-07-01,Spouse,--,JNL/JPMorgan USGov&Qual,"$1,001 - $15,000","Coons, Chris",Purchase
34637,2002-07-01,Spouse,--,JNL/JPMorgan MidCapGrowth,"$1,001 - $15,000","Coons, Chris",Purchase
25684,2002-12-07,Spouse,VOX,Vanguard Communication Services ETF,"$1,001 - $15,000","Whitehouse, Sheldon",Sale (Full)
26694,2002-12-07,Spouse,VOX,Vanguard Communication Services ETF,"$1,001 - $15,000","Whitehouse, Sheldon",Sale (Full)
14314,2012-01-02,Self,WAG,Walgreen Co. (NYSE),"$1,001 - $15,000",The Honorable Thad Cochran (Former Senator),Purchase


#### Uh oh
It looks like the first 5 entrys have a transaction year of 2002, well before the earliest reporting year of 2013. It is most likely a typo in the senate website. 
We will just remove these entries as it is not possible to tell when the trades actually take place.

In [298]:
senator_filing_data_df.drop([30104,34628,34637,25684,26694], inplace=True)

In [299]:
sorted_filings = senator_filing_data_df.sort_values(by='Transaction Date', ascending=True)
sorted_filings.head(5)

,Transaction Date,Owner,Ticker,Asset Name,Amount,Senator Name,Transaction Type
1938,2012-01-02,Self,WAG,Walgreen Co. (NYSE),"$1,001 - $15,000",The Honorable Thad Cochran (Former Senator),Purchase
14314,2012-01-02,Self,WAG,Walgreen Co. (NYSE),"$1,001 - $15,000",The Honorable Thad Cochran (Former Senator),Purchase
2185,2012-01-02,Self,WAG,Walgreen Co. (NYSE),"$1,001 - $15,000",The Honorable Thad Cochran (Former Senator),Purchase
2782,2012-01-02,Self,WAG,Walgreen Co. (NYSE),"$1,001 - $15,000",The Honorable Thad Cochran (Former Senator),Purchase
14315,2012-02-22,Self,DPS,"Dr Pepper Snapple Group, Inc. (NYSE)","$1,001 - $15,000",The Honorable Thad Cochran (Former Senator),Purchase


Now lets clean up the Senator Name column. The scraper I wrote is not very robust when it comes to collecting names of senators, so in the case that the senator was terminated at the time of filing, the scraper grabbed the entire header tag containing the senators name, ie "The Honorable Thad Cochran (Former Senator)." Additionally, its possible that some entries have the "(Former Senator)" string but not the "The Honorable" sring. Finally, in some cases the name may include "Mr", "Ms", or "Mrs" such as "Ms. Kamala D Harris (Former Senator)." Lets get rid of the unecessary stuff, and put the name in LAST, FIRST format to match the rest of the entries. 

In [300]:
mask = senator_filing_data_df['Senator Name'].str.contains("Former Senator")
# Change the entries in the original DataFrame based on the mask

def fix_senator_name(name_entry):
    pattern = r"(The Honorable|Ms.|Mr.|Mrs.)? ([a-zA-Z]*)?\s*([A-Z])? ([a-zA-Z]*)?\s*(Jr)? (\(Former Senator\))?"
        
    matches = re.findall(pattern, name_entry) 
    #Get rid of title cap group ie "Mr", "The Honorable" etc
    #Also get rid of "Former Senator"
    match = matches[0]

    f_name = ''
    l_name = ''

    if (len(match) == 6):
        if (match[2]):
            f_name = match[1] + " " + match[2]
        else:
            f_name = match[1] + match[2]
            
        if (match[4]):
            l_name = match[3] + " " + match[4]
        else:
            l_name = match[3] + match[4]
        
        name = l_name + ', ' + f_name
        return name
        
senator_filing_data_df.loc[mask, 'Senator Name'] = senator_filing_data_df.loc[mask, 'Senator Name'].apply(fix_senator_name)

In [301]:
senator_filing_data_df.head(5)

,Transaction Date,Owner,Ticker,Asset Name,Amount,Senator Name,Transaction Type
0,2014-04-29,Joint,LINE,"Linn Energy, LLC (NASDAQ)","$1,001 - $15,000","Toomey, Patrick J",Sale (Full)
1,2014-06-05,Joint,GLD,SPDR Gold Shares,"$1,001 - $15,000","Toomey, Patrick J",Sale (Partial)
2,2014-06-05,Joint,PPL,PPL Corporation (NYSE),"$15,001 - $50,000","Toomey, Patrick J",Sale (Partial)
3,2014-06-10,Self,--,Team Capitol Bank,"$500,001 - $1,000,000","Toomey, Patrick J",Exchange
4,2014-06-11,Joint,LGP,Lehigh Gas Partners LP (NYSE),"$15,001 - $50,000","Toomey, Patrick J",Purchase


In [302]:
blank_ticker = senator_filing_data_df[senator_filing_data_df['Ticker'] == '--']
blank_ticker.head(5)

,Transaction Date,Owner,Ticker,Asset Name,Amount,Senator Name,Transaction Type
3,2014-06-10,Self,--,Team Capitol Bank,"$500,001 - $1,000,000","Toomey, Patrick J",Exchange
51,2014-08-07,Joint,--,Lehigh gass Partners LP UT,"$15,001 - $50,000","Toomey, Patrick J",Sale (Full)
55,2014-06-11,Joint,--,Lehigh Gas Partners LP UT,"$15,001 - $50,000","Toomey, Patrick J",Purchase
102,2014-01-02,Spouse,--,11.15% due 9/30/16 RBC Autocall Blackstone,"$15,001 - $50,000","Carper, Thomas R.",Sale (Full)
103,2014-01-03,Spouse,--,8.6% due 1/8/15 HSBC Autocall Broadcom,"$15,001 - $50,000","Carper, Thomas R.",Purchase


It looks like some entries where the ticker is equal to '--' include things that arent publicly traded stocks, such as retiremnet investment accounds, college fund portfolios, annuities, etc. Lets get rid of them because we wont be able to easily find stock information on these purchases. 

In [303]:
senator_filing_data_df = senator_filing_data_df[~(senator_filing_data_df['Ticker'] == '--')]

Additionally,lets remove the rows where the transcation type is "Exchange" as we wont be able to gain any information from them

In [310]:
senator_filing_data_df = senator_filing_data_df[~(senator_filing_data_df['Transaction Type'] == 'Exchange')]

41178
40957


Possibly combine so each transaction type is in one row

In [304]:
senator_filing_data_df.head()

,Transaction Date,Owner,Ticker,Asset Name,Amount,Senator Name,Transaction Type
0,2014-04-29,Joint,LINE,"Linn Energy, LLC (NASDAQ)","$1,001 - $15,000","Toomey, Patrick J",Sale (Full)
1,2014-06-05,Joint,GLD,SPDR Gold Shares,"$1,001 - $15,000","Toomey, Patrick J",Sale (Partial)
2,2014-06-05,Joint,PPL,PPL Corporation (NYSE),"$15,001 - $50,000","Toomey, Patrick J",Sale (Partial)
4,2014-06-11,Joint,LGP,Lehigh Gas Partners LP (NYSE),"$15,001 - $50,000","Toomey, Patrick J",Purchase
5,2014-06-12,Joint,DBP,PowerShares DB Precious Metals,"$15,001 - $50,000","Toomey, Patrick J",Purchase


Lets see how many unique stocks there are.

In [311]:
unique_stocks = senator_filing_data_df['Ticker'].unique()
print(f"Number of Filings: {senator_filing_data_df.shape[0]}")
print(f"Number of Unique Stocks: {len(unique_stocks)}")



Number of Filings: 40957
Number of Unique Stocks: 3353


Wow! Of the 40957 senator filings from 2012 to 2024, there are only 3353 unique stock purchases.

### Obtain The Stock Information:
In this section, we'll need to collect information on each of the 3432 stocks. We'll use the yahoo_fin package to collect 
the information from Yahoo Finance. It is important to note that yahoo_fin is free, but may not have the most accurate data

In [ ]:
!pip install yahoo_fin

Here is a demo of yahoo_fin.

In [ ]:
from yahoo_fin.stock_info import get_data

amazon_weekly= get_data("line")
amazon_weekly.head()

yahoo_fin returns a pandas df containing the stock information at specified intervals, in this case it contains daily OHLCV data. Lets collect a dataframe for each stock and store it in a dictionary. NOTE, some companies aren't available in yahoo finance for various reasons, for example Linn Energy, LLC (LINE) sold by Senator Patrick Toomey went bankrupt in 2016. While information on the stocks is useful for understanding the big picture of senators trading patterns and possible insight they have (Toomey Sold before the company went bankrupt), it is not feasible to collect this information without spending more time and money to retrieve this information. In this case, we will just skip these stocks.

historical_datas = {}
for i,ticker in enumerate(unique_stocks):
    try:
        historical_datas[ticker] = get_data(ticker, start_date="1/01/2011", interval="1d")
    except:
        print(f"stock {ticker} not found")

In [346]:
print(f'Number of Stock Tables: {len(historical_datas)}')
print(f'Data Lost: {len(unique_stocks)-len(historical_datas)}')

Number of Stock Tables: 2982
Data Lost: 371


Looks like we werent able to find data on 371 stocks. Lets drop those from the senator_filing_data_df as they will be of no use to us.

In [398]:
#Contain a giant string of all historical_datas.keys() combined


unique_stocks_set = set(unique_stocks)
historical_datas_keys_set = set(historical_datas.keys())

missing_stock_data = unique_stocks_set - (historical_datas_keys_set)
mask = senator_filing_data_df['Ticker'].isin(missing_stock_data)

#sanity check
#This df contains all entries with unknown tickers
#The unique values should total to 371
illegal_ticker = senator_filing_data_df[mask]["Ticker"].unique()
illegal_ticker.shape

#Remove from senator_filing_data_df
senator_filing_data_df = senator_filing_data_df[~mask]
#Should be 2982
senator_filing_data_df["Ticker"].unique().shape

(2982,)

Now that the senator_filing_data_df df and the dictonary of historical stock data is all squared away, lets visualize some data. 

## Exploratory Analysis and Data Visualization

Lets first start off by visualizing the performence of each of the 2982 stocks from 2012 to today.

Lets identify the top performing stocks and graph them over time

Now that weve seen how the stocks performed, lets see how the senators did. For this